In [180]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [181]:
pip install webdriver_manager


Note: you may need to restart the kernel to use updated packages.


In [182]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

In [183]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [184]:
driver = webdriver.Chrome(ChromeDriverManager().install())
def login():   
    url = 'https://www.coderbyte.com/solution/Bracket%20Matcher'
    userName = 'nnnnb'
    password = '12334'
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_class_name("login-link").click()
    time.sleep(2)
    driver.find_element_by_class_name("mode-toggle-option").click()
    time.sleep(1)
    driver.find_element_by_class_name("login-field-input").send_keys(userName)
    time.sleep(1)
    driver.find_element_by_css_selector("input[type='password']").send_keys(password)
    time.sleep(1)
    xpath = '//*[@id="login"]/div/div/div/div[1]/div[3]/button'
    driver.find_element_by_xpath(xpath).click()
    time.sleep(2)


[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\Noam\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


In [185]:
#login()

In [209]:
def selectLang(lang):
    url = 'https://www.coderbyte.com/solution/Bracket%20Matcher'
    driver.get(url)
    time.sleep(3)

    xpath = '//*[@id="challenge_leaderboard"]/div/div/div[2]/div/div[2]/span'
    driver.find_element_by_xpath(xpath).click()
    time.sleep(1)
    driver.find_element_by_css_selector("input[type='text']").send_keys(lang)
    time.sleep(1)
    driver.find_element_by_css_selector("input[type='text']").send_keys(Keys.ENTER)

    
def data_process(html):
    clean_data_list = []

    soup = BeautifulSoup(html, 'html.parser')
    cards = soup.find_all("div", {"class": "comment card"})

    for card in cards:
        data_dict = {}
        sol_code = card.find("code",{"class":"hljs"})
        sol_card = card.find("div",{"class":"submitter"})
        data_dict['user_url'] = sol_card.find('a')['href']
        data_dict['user_img'] = sol_card.find('img')['src']
        data_dict['user_score'] = sol_card.find('strong').text.strip()
        data_dict['sol_Code'] = sol_code
        data_dict['user_name'] = sol_card.find('a').text
        clean_data_list.append(data_dict)
        
    return clean_data_list

In [210]:
lang_name = 'Java'
selectLang(lang_name)

In [211]:
html = driver.page_source
clean_data_list = data_process(html)

In [212]:
clean_data_list

[{'user_url': '/profile/arcavious',
  'user_img': 'https://s3-us-west-1.amazonaws.com/coderbyteprofiles/arcavious.jpg',
  'user_score': '10',
  'sol_Code': <code class="hljs javascript"><span class="hljs-keyword">import</span> java.util.*; 
  <span class="hljs-keyword">import</span> java.io.*;
  <span class="hljs-keyword">import</span> java.util.regex.Matcher;
  <span class="hljs-keyword">import</span> java.util.regex.Pattern;
  
  <span class="hljs-class"><span class="hljs-keyword">class</span> <span class="hljs-title">Function</span> </span>{  
    int BracketMatcher(<span class="hljs-built_in">String</span> str) { 
    
      <span class="hljs-comment">// code goes here   </span>
      <span class="hljs-comment">/* Note: In Java the return type of a function and the 
         parameter types being passed are defined, so this return 
         call must match the return type of the function.
         You are free to modify the return type. */</span>
         
      Pattern p = Pattern